# 유방암 분류를 위한 PyTorch 신경망 모델

이 노트북은 유방암 데이터셋을 사용하여 이진 분류를 수행하는 PyTorch 신경망 모델을 구현합니다.

## 주요 내용
1. 데이터 전처리 및 로드
2. 신경망 모델 정의
3. 모델 훈련
4. 모델 평가

## 1. 필요한 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

## 2. 신경망 모델 정의

`CancerClassifier` 클래스는 유방암 분류를 위한 신경망 모델을 정의합니다.

In [ ]:
class CancerClassifier(nn.Module):
    """유방암 분류를 위한 신경망 모델"""
    
    def __init__(self, input_size=30, hidden_sizes=[64, 32], output_size=1):
        """
        모델 초기화
        Args:
            input_size (int): 입력 특성 수 (기본값: 30)
            hidden_sizes (list): 은닉층 크기 리스트 (기본값: [64, 32])
            output_size (int): 출력 크기 (기본값: 1)
        """
        super(CancerClassifier, self).__init__()
        
        layers = []
        prev_size = input_size
        
        # 은닉층 생성
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))  # 선형 변환
            layers.append(nn.ReLU())  # 활성화 함수
            prev_size = hidden_size
        
        # 출력층 추가
        layers.append(nn.Linear(prev_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        """순전파 함수"""
        return self.network(x)

## 3. 데이터 전처리 클래스

`CancerDataProcessor` 클래스는 유방암 데이터의 전처리 및 DataLoader 생성을 담당합니다.

In [ ]:
class CancerDataProcessor:
    """유방암 데이터 전처리 및 로더 생성 클래스"""
    
    def __init__(self, test_size=0.2, batch_size=32):
        """
        데이터 처리기 초기화
        Args:
            test_size (float): 테스트 데이터 비율 (기본값: 0.2)
            batch_size (int): 배치 크기 (기본값: 32)
        """
        self.test_size = test_size
        self.batch_size = batch_size
        self.scaler = StandardScaler()  # 데이터 정규화를 위한 스케일러
    
    def load_and_prepare_data(self):
        """데이터 로드 및 전처리"""
        # 유방암 데이터셋 로드
        cancer = load_breast_cancer()
        # 데이터 정규화 (평균 0, 표준편차 1)
        X = self.scaler.fit_transform(cancer.data)
        y = cancer.target
        
        # 훈련/테스트 데이터 분할 (계층화 샘플링 사용)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, stratify=y, random_state=42
        )
        
        return self._create_dataloaders(X_train, X_test, y_train, y_test)
    
    def _create_dataloaders(self, X_train, X_test, y_train, y_test):
        """PyTorch 텐서로 변환 및 데이터로더 생성"""
        # NumPy 배열을 PyTorch 텐서로 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
        
        # 데이터셋 생성
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        
        # 데이터로더 생성 (배치 단위로 데이터 제공)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
        
        return train_loader, test_loader

## 4. 모델 훈련 및 평가 클래스

`CancerTrainer` 클래스는 모델의 훈련과 평가를 담당합니다.

In [ ]:
class CancerTrainer:
    """모델 훈련 및 평가를 담당하는 클래스"""
    
    def __init__(self, model, learning_rate=0.001):
        """
        훈련기 초기화
        Args:
            model: 훈련할 모델
            learning_rate (float): 학습률 (기본값: 0.001)
        """
        self.model = model
        self.criterion = nn.BCEWithLogitsLoss()  # 이진 분류용 손실 함수
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam 옵티마이저
    
    def train(self, train_loader, epochs=100):
        """
        모델 훈련
        Args:
            train_loader: 훈련 데이터로더
            epochs (int): 훈련 에포크 수 (기본값: 100)
        """
        self.model.train()  # 훈련 모드 설정
        
        for epoch in range(epochs):
            total_loss = 0
            
            # 배치별 훈련
            for inputs, labels in train_loader:
                self.optimizer.zero_grad()  # 그래디언트 초기화
                outputs = self.model(inputs)  # 순전파
                loss = self.criterion(outputs, labels)  # 손실 계산
                loss.backward()  # 역전파
                self.optimizer.step()  # 가중치 업데이트
                total_loss += loss.item()
            
            # 20 에포크마다 손실 출력
            if (epoch + 1) % 20 == 0:
                avg_loss = total_loss / len(train_loader)
                print(f'Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}')
        
        print("Training completed!")
    
    def evaluate(self, test_loader):
        """
        모델 평가
        Args:
            test_loader: 테스트 데이터로더
        Returns:
            float: 정확도
        """
        self.model.eval()  # 평가 모드 설정
        correct = 0
        total = 0
        
        # 그래디언트 계산 비활성화 (메모리 절약)
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = self.model(inputs)  # 예측
                # 시그모이드 함수 적용 후 반올림하여 0 또는 1로 변환
                predicted = torch.round(torch.sigmoid(outputs))
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        # 정확도 계산 및 출력
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')
        return accuracy

## 5. 모델 실행

이제 정의한 클래스들을 사용하여 유방암 분류 모델을 훈련하고 평가해보겠습니다.

In [ ]:
# 데이터 전처리
data_processor = CancerDataProcessor(test_size=0.2, batch_size=32)
train_loader, test_loader = data_processor.load_and_prepare_data()

print(f"훈련 데이터 배치 수: {len(train_loader)}")
print(f"테스트 데이터 배치 수: {len(test_loader)}")

In [ ]:
# 모델 생성
model = CancerClassifier(input_size=30, hidden_sizes=[64, 32], output_size=1)
print(f"모델 구조:\n{model}")

In [ ]:
# 훈련
trainer = CancerTrainer(model, learning_rate=0.001)
trainer.train(train_loader, epochs=100)

In [ ]:
# 평가
accuracy = trainer.evaluate(test_loader)
print(f"\n최종 테스트 정확도: {accuracy:.2f}%")

## 6. 결과 분석

이 모델은 유방암 데이터셋을 사용하여 이진 분류를 수행합니다. 주요 특징:

- **입력**: 30개의 특성 (유방암 데이터셋의 특성 수)
- **아키텍처**: 30 → 64 → 32 → 1 (ReLU 활성화 함수 사용)
- **손실 함수**: BCEWithLogitsLoss (이진 분류에 적합)
- **옵티마이저**: Adam (적응적 학습률)
- **데이터 전처리**: StandardScaler로 정규화
- **평가 지표**: 정확도 (Accuracy)

일반적으로 이 모델은 90% 이상의 높은 정확도를 달성할 수 있습니다.